### Cross Convolution

In [2]:
from typing import Optional, Tuple, Union

import einops as E
import torch
import torch.nn as nn
from pydantic import validate_arguments

In [3]:
size2t = Union[int, Tuple[int, int]]

- Dòng code trên đang định nghĩa 1 biến mới (Ví dụ: int là số nguyên).
- Biến size2t được định nghĩa có thể là số nguyên hoặc 1 tuple chứa 2 số nguyên.
- Nó như 1 hướng dẫn là size2t nên là int hoặc tuple (int,int)

In [13]:
# Định nghĩa một hàm sử dụng biến size2t
size2t = Union[int, Tuple[int, int]]
def process_size(size: str):
    print(size)

# Sử dụng hàm với các đối số khác nhau
process_size(10)                # In: Processing a single integer: 10
process_size((20, 30))           # In: Processing a tuple of two integers: 20 and 30
process_size("abc")   # In: Invalid size format

10
(20, 30)
abc


Hàm code trên thay kiểu dữ liệu của size khác int hoặc str cũng k bị lỗi
-> Nó chỉ là hướng dẫn biến đó nên là loại dữ liệu nào.

In [37]:
class CrossConv2d(nn.Conv2d):
    """
    Thực hiện tích chập theo cặp giữa tất cả các phần tử của x và tất cả các phần tử của y.
    x, y là các tensor có kích thước B,_,C,H,W, trong đó _ có thể là số lượng khác nhau của các phần tử trong x và y.
    Cụ thể, chúng ta thực hiện một meshgrid của các phần tử để có được các tensor B, Sx, Sy, C, H, W, và sau đó
    thực hiện tích chập theo cặp.
    Args:
        x (tensor): B,Sx,Cx,H,W
        y (tensor): B,Sy,Cy,H,W
    Returns:
        tensor: B,Sx,Sy,Cout,H,W
    """

    """
    Parameters
    ----------
    in_channels : int hoặc tuple của ints
    Số kênh trong tensor đầu vào (các).
    Nếu các tensor có số lượng kênh khác nhau, in_channels phải là một tuple
    out_channels : int
        Số lượng kênh đầu ra.
    kernel_size : int hoặc tuple của ints
        Kích thước của kernel tích chập.
    stride : int hoặc tuple của ints, tùy chọn
        Bước của tích chập. Mặc định là 1.
    padding : int hoặc tuple của ints, tùy chọn
        Zero-padding được thêm vào cả hai bên của đầu vào. Mặc định là 0.
    dilation : int hoặc tuple của ints, tùy chọn
        Khoảng cách giữa các phần tử của kernel. Mặc định là 1.
    groups : int, tùy chọn
        Số lượng kết nối chặn từ các kênh đầu vào đến các kênh đầu ra. Mặc định là 1.
    bias : bool, tùy chọn
        Nếu True, thêm một bias có thể học được vào đầu ra. Mặc định là True.
    padding_mode : str, tùy chọn
        Chế độ đổ đầy. Mặc định là "zeros".
    device : str, tùy chọn
        Thiết bị mà tensor được phân bổ. Mặc định là None.
    dtype : torch.dtype, tùy chọn
        Loại dữ liệu được gán cho tensor. Mặc định là None.

    Returns
    -------
    torch.Tensor
        Tensor kết quả từ tích chập theo cặp giữa các phần tử của x và y.

    Notes
    -----
    x và y là các tensor có kích thước (B, Sx, Cx, H, W) và (B, Sy, Cy, H, W), tương ứng,
    Hàm thực hiện tích chập theo cặp của các phần tử của x và y để có được một tensor
    có kích thước (B, Sx, Sy, Cx + Cy, H, W), và sau đó thực hiện tích chập tương tự cho tất cả
    (B, Sx, Sy) trong chiều batch. Thời gian chạy và bộ nhớ là O(Sx * Sy).

    Examples
    --------
    >>> x = torch.randn(2, 3, 4, 32, 32)
    >>> y = torch.randn(2, 5, 6, 32, 32)
    >>> conv = CrossConv2d(in_channels=(4, 6), out_channels=7, kernel_size=3, padding=1)
    >>> output = conv(x, y)
    >>> output.shape  #(2, 3, 5, 7, 32, 32)
    """

    @validate_arguments
    def __init__(
        self,
        in_channels: size2t,
        out_channels: int,
        kernel_size: size2t,
        stride: size2t = 1,
        padding: size2t = 0,
        dilation: size2t = 1,
        groups: int = 1,
        bias: bool = True,
        padding_mode: str = "zeros",
        device=None,
        dtype=None,
    ) -> None:

        if isinstance(in_channels, (list, tuple)):
            concat_channels = sum(in_channels)
        else:
            concat_channels = 2 * in_channels

        super().__init__(
            in_channels=concat_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            groups=groups,
            bias=bias,
            padding_mode=padding_mode,
            device=device,
            dtype=dtype,
        )

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        """
        Compute pairwise convolution between all elements of x and all elements of y.

        Parameters
        ----------
        x : torch.Tensor
            Input tensor of size (B, Sx, Cx, H, W).
        y : torch.Tensor
            Input tensor of size (B, Sy, Cy, H, W).

        Returns
        -------
        torch.Tensor
            Tensor resulting from the cross-convolution between the elements of x and y.
            Has size (B, Sx, Sy, Co, H, W), where Co is the number of output channels.
        """
        B, Sx, *_ = x.shape # B: Số batch, Số ảnh trong 1 batch, kích thước từng ảnh (3 chiều C,H,W) 
        _, Sy, *_ = y.shape # Tương tự
        # Chỉ lấy 3 chỉ số B, Sx và Sy
        xs = E.repeat(x, "B Sx Cx H W -> B Sx Sy Cx H W", Sy=Sy) # Tạo meshgrid
        ys = E.repeat(y, "B Sy Cy H W -> B Sx Sy Cy H W", Sx=Sx)
        print("xs: ",xs.shape)
        print("ys: ",ys.shape)
        xy = torch.cat([xs, ys], dim=3,)
        print("xy: ",xy.shape)
        batched_xy = E.rearrange(xy, "B Sx Sy C2 H W -> (B Sx Sy) C2 H W")
        print("batched_xy: ", batched_xy.shape)
        batched_output = super().forward(batched_xy)
        print("batched_output: ", batched_output.shape)
        output = E.rearrange(
            batched_output, "(B Sx Sy) Co H W -> B Sx Sy Co H W", B=B, Sx=Sx, Sy=Sy
        )
        print(output.shape)
        return output


In [38]:
# Create sample tensors
x = torch.randn(2, 3, 4, 32, 32)  # B=2, Sx=3, Cx=4, H=32, W=32
y = torch.randn(2, 5, 6, 32, 32)  # B=2, Sy=5, Cy=6, H=32, W=32

# Instantiate CrossConv2d
conv = CrossConv2d(in_channels=(4, 6), out_channels=7, kernel_size=3, padding=1)

# Pass through the forward method
output = conv(x, y)

# Print the shape of the output
print(output)

xs:  torch.Size([2, 3, 5, 4, 32, 32])
ys:  torch.Size([2, 3, 5, 6, 32, 32])
xy:  torch.Size([2, 3, 5, 10, 32, 32])
batched_xy:  torch.Size([30, 10, 32, 32])
batched_output:  torch.Size([30, 7, 32, 32])
torch.Size([2, 3, 5, 7, 32, 32])
tensor([[[[[[-1.2668e-01, -1.1445e-03,  2.4181e-01,  ..., -1.5604e-01,
             -4.2923e-01, -8.8293e-01],
            [ 1.1165e+00, -1.2571e-01, -1.6441e-01,  ...,  7.2219e-01,
             -4.6175e-01, -4.8763e-02],
            [ 1.4007e-01,  5.5220e-02,  2.6847e-01,  ..., -3.5917e-03,
              2.3202e-01,  2.5519e-01],
            ...,
            [ 7.3286e-01, -1.5096e+00, -1.2910e-01,  ...,  8.3857e-01,
              5.2697e-01, -3.1364e-01],
            [ 2.3299e-01, -5.8996e-02, -8.3288e-01,  ...,  3.1654e-01,
             -6.9337e-02, -1.1619e-01],
            [ 6.9594e-01, -5.2538e-01,  2.5318e-01,  ...,  1.1251e-01,
             -5.5924e-01, -3.7271e-01]],

           [[-7.1158e-01, -1.0544e-01, -9.8701e-01,  ...,  9.7943e-01,
         

#### Giải thích
- 2 tensors: (2, 3, 4, 32, 32) và (2, 5, 6, 32, 32)
- Tạo 1 layer CrossConv2d
    - Số kênh (C) của x và y (4,6)
    - Số kênh (C) output
    - kernel_size = 3
    - padding = 1 (Có thêm 1 viền ngoài, phần này kế thừa lớp conv2D bth)
- xs: Lặp lại x theo chiều Sy, tạo ra tensor có kích thước (2, 3, 5, 4, 32, 32).
- ys: Lặp lại y theo chiều Sx, tạo ra tensor có kích thước (2, 3, 5, 6, 32, 32).
- Nối lại theo chiều kênh (2, 3, 5, 10, 32, 32)
- Sắp xếp lại xy để có kích thước ((2 * 3 * 5), 10, 32, 32), kết hợp batch và chiều không gian (Sx, Sy).
- Qua lớp conv output cuối cùng là 7: [30, 7, 32, 32]
- output trả về chiều như cũ

- Vì cảm thấy không cần hiểu cấu trúc từng bước, chỉ làm input, output và cách ảnh đi qua từng lớp 1